In [36]:
import random
import re
import string
from collections import Counter
import math
import pandas as pd

class Wordle(object):

    def __init__(self):
        self._answers = self.readfiles('2315_possible_answers.txt')
        self._allowed = self.readfiles('12972_allowed_words.txt')
        self._allowedWordEntropy = {line.split()[0] : float(line.split()[1])  for line in self.readfiles('12972_allowed_words_entroy.txt')}
        self._passed = self._allowed
        self.wordprob = pd.read_csv('word_frequency.csv', index_col = ['word'])
        self.wordprob = self.wordprob.T.to_dict('r')[0]
        self.best = { key: value for key, value in self._allowedWordEntropy.items() if value > 6}



    #----   Reading Files -------

    def readfiles(self, file , entropy = False):
        with open(file,'r') as fh:
            lines = fh.readlines()
        return [line.rstrip('\n') for line in lines]

    #----- Green Gray or Yellow
    def isMatched(self,text, pattern):
        if re.search(pattern, text):
            return True
        return False
    
    def gray(self, grayset, allowed = None):
        if allowed is None:
            allowed = self._passed

        if grayset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(grayset))
        passed_ = []
        for word in allowed:
            if not self.isMatched(word, pattern):
                passed_.append(word)
        
        return passed_


    def green(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            greenList = list(greenList)
            char , position = greenList[0]
            for word in allowed:
                if word[position] == char:
                    passed_.append(word)
            return self.green(greenList[1:],passed_)
    
    def notGreen(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            char , position = greenList[0]
            for word in allowed:
                if word[position] != char:
                    passed_.append(word)
            return self.notGreen(greenList[1:],passed_)

    def yellow(self, yellowList,yellowset,allowed):

        if yellowset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(yellowset))
        passed_ = []
        for word in allowed:
            if self.isMatched(word, pattern):
                passed_.append(word)
        passed_ = self.notGreen(yellowList,passed_)
        return passed_

    #---- WordleWord Test

    def compareWord(self,guess,word, tup = False):
        gray_ = ""
        yellow_ = ""
        yellowList_ = set()
        greenList_ = set()
        
        for i in range(0,5):
            if guess[i] == word[i]:
                greenList_.add((guess[i],i))
            else:
                if guess[i] in word:
                    if guess[i] not in yellow_:
                        yellow_ += guess[i]
                    yellowList_.add((guess[i],i))
                if guess[i] not in word and guess[i] not in gray_:
                    gray_ += guess[i]
        
        if tup:
            return (gray_, tuple(yellowList_), tuple(greenList_))
        else:
            return {    "gray"      : gray_ ,
                    "yellow"    : yellow_,
                    "yellowList" : yellowList_,
                    "greenList" : greenList_ }

    
    def bestGuess(self,allowed,chance):
        if len(allowed) == 1:
            return allowed.pop()

        if chance == 6:
            guess = random.choices(list(self.best.keys()), weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
            return guess
        elif chance == 5:
            best = {key: self._allowedWordEntropy[key] for key in allowed}
            guess = max(best, key= best.get)
            return guess
        if chance > 3:
            try:
                guess = random.choices(allowed, weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
                return guess
            except ValueError:
                guess = random.choice(allowed)
        elif chance > 0:
            if chance == 3:
                self.updateProba(allowed)
            try:
                guess = random.choices(allowed, weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
            except ValueError:
                guess = random.choice(allowed)
            return guess
        else:
            guess = random.choice(allowed)
            return guess
        

    def reduce(self, guess_word,allowed):
        meta = self.compareWord(guess_word,self._wordleWord)
        allowed = self.yellow(list(meta["yellowList"]),meta['yellow'],allowed)
        allowed = self.gray(meta['gray'],allowed)
        allowed = self.green(list(meta['greenList']),allowed)
        #print(len(allowed))
        return allowed

    # ---- Remembering Old
    def calEntropy(self,wordprob):
        entropy = {} 
        for key,value in wordprob.items():
            entropy.update({key : value * math.log2(1/value)})
        return entropy

    def letRemember(self):
        try:
            with open("word_frequency.txt","a") as fh:
                fh.write(self._wordleWord + "\n")

        except AttributeError:
            pass

    def updateProba(self,allowed, update = False):
        
        #print(wordprob)
        for key in allowed:
            self._allowedWordEntropy[key] *= self.wordprob[key]
        return None
    
    # ----- Run

    def run(self,chance = 6, train = False):
        self._wordleWord = random.choice(self._answers)
        #print(self._wordleWord.upper())

        allowed = self._passed
        while len(allowed) != 0:
            if chance == 0:
                self._allowedWordEntropy = {line.split()[0] : float(line.split()[1])  for line in self.readfiles('12972_allowed_words_entroy.txt')}
                if train:
                    self.letRemember()
                return 0, 0
                re
            chance -= 1
            guess_word = self.bestGuess(allowed,chance)
            #print(guess_word.upper(), end=' ')
            if guess_word == self._wordleWord:
                self._allowedWordEntropy = {line.split()[0] : float(line.split()[1])  for line in self.readfiles('12972_allowed_words_entroy.txt')}
                return 1, chance
            allowed = self.reduce(guess_word, allowed)


In [37]:
mn,mx,avg,count = 100,0,[],0
gm = Wordle()
for i in range(10):
    
    count += 1
    run = 200
    passed = 0
    chance = 0
    for i in range(run):
        p , c = gm.run()
        if p:
            passed += 1
            chance += (6 - c)
    accuracy = round(passed/run * 100,2)
    if mn > accuracy:
        mn = accuracy
    if mx < accuracy:
        mx = accuracy
    avg.append(accuracy)
    print("{5}. Accuracy : {0}% , Average Chance : {1}, Min Accuracy: {2}, Max Accuracy: {3} , Mean Accuracy: {4} ".format(accuracy,chance/run,mn,mx,sum(avg)/count,count))

C:\Users\etuca\AppData\Local\Temp/ipykernel_2308/27113907.py:16: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  self.wordprob = self.wordprob.T.to_dict('r')[0]


1. Accuracy : 94.5% , Average Chance : 4.12, Min Accuracy: 94.5, Max Accuracy: 94.5 , Mean Accuracy: 94.5 
2. Accuracy : 91.5% , Average Chance : 3.955, Min Accuracy: 91.5, Max Accuracy: 94.5 , Mean Accuracy: 93.0 
3. Accuracy : 88.5% , Average Chance : 3.83, Min Accuracy: 88.5, Max Accuracy: 94.5 , Mean Accuracy: 91.5 
4. Accuracy : 92.0% , Average Chance : 3.955, Min Accuracy: 88.5, Max Accuracy: 94.5 , Mean Accuracy: 91.625 


KeyboardInterrupt: 